## Initialization

In [ ]:
#@title
! git clone https://github.com/proroklab/VectorizedMultiAgentSimulator.git
%cd /content/VectorizedMultiAgentSimulator
!pip install -e .

In [ ]:
#@title
!sudo apt-get update
!sudo apt-get install python3-opengl xvfb
!pip install pyvirtualdisplay
import pyvirtualdisplay
display = pyvirtualdisplay.Display(visible=False, size=(1400, 900))
display.start()

## Run


In [ ]:
from vmas.simulator.scenario import BaseScenario
from typing import Union
import time
import torch
from vmas import make_env

def use_vmas_env(
    render: bool = False,
    save_render: bool = False,
    num_envs: int = 32,
    n_steps: int = 100,
    device: str = "cpu",
    scenario: Union[str, BaseScenario]= "waterfall",
    n_agents: int = 4,
    continuous_actions: bool = True,
    **kwargs
):
    """Example function to use a vmas environment

    Args:
        continuous_actions (bool): Whether the agents have continuous or discrete actions
        n_agents (int): Number of agents
        scenario (str): Name of scenario
        device (str): Torch device to use
        render (bool): Whether to render the scenario
        save_render (bool):  Whether to save render of the scenario
        num_envs (int): Number of vectorized environments
        n_steps (int): Number of steps before returning done

    Returns:

    """
    assert not (save_render and not render), "To save the video you have to render it"

    simple_2d_action = (
        [0, -1.0] if continuous_actions else [3]
    )  # Simple action for an agent with 2d actions

    scenario_name = scenario if isinstance(scenario,str) else scenario.__class__.__name__

    env = make_env(
        scenario=scenario,
        num_envs=num_envs,
        device=device,
        continuous_actions=continuous_actions,
        wrapper=None,
        seed=None,
        # Environment specific variables
        n_agents=n_agents,
        **kwargs
    )

    frame_list = []  # For creating a gif
    init_time = time.time()
    step = 0

    for s in range(n_steps):
        step += 1
        print(f"Step {step}")

        actions = []
        for i, agent in enumerate(env.agents):
            action = torch.tensor(
                simple_2d_action,
                device=device,
            ).repeat(num_envs, 1)

            actions.append(action)

        obs, rews, dones, info = env.step(actions)

        if render:
            frame = env.render(
                mode="rgb_array" if save_render else "human",
                agent_index_focus=None,  # Can give the camera an agent index to focus on
                visualize_when_rgb=True,
            )
            if save_render:
                frame_list.append(frame)

    total_time = time.time() - init_time
    print(
        f"It took: {total_time}s for {n_steps} steps of {num_envs} parallel environments on device {device} "
        f"for {scenario_name} scenario."
    )

    if render and save_render:
        from moviepy.editor import ImageSequenceClip
        fps=30
        clip = ImageSequenceClip(frame_list, fps=fps)
        clip.write_gif(f'{scenario_name}.gif', fps=fps)

In [ ]:
scenario_name="waterfall"
use_vmas_env(
    scenario=scenario_name,
    render=True,
    save_render=True,
    num_envs=32,
    n_steps=150,
    device="cuda",
    continuous_actions=True,
    # Environment specific variables
    n_agents=4,
)

In [ ]:
from IPython.display import Image
Image(f'{scenario_name}.gif')